In [40]:
import pandas as pd
import numpy as np

df = pd.read_csv("heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [41]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [105]:
df.describe()
df.shape

(918, 12)

In [43]:
cols = ["Age","RestingBP","Cholesterol","FastingBS","MaxHR","Oldpeak"]

In [44]:
cols

['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']

In [45]:
mean = df[cols].mean()
mean

Age             53.510893
RestingBP      132.396514
Cholesterol    198.799564
FastingBS        0.233115
MaxHR          136.809368
Oldpeak          0.887364
dtype: float64

In [46]:
std_dev =  df[cols].std()
std_dev

Age              9.432617
RestingBP       18.514154
Cholesterol    109.384145
FastingBS        0.423046
MaxHR           25.460334
Oldpeak          1.066570
dtype: float64

In [47]:
df[cols].head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak
0,40,140,289,0,172,0.0
1,49,160,180,0,156,1.0
2,37,130,283,0,98,0.0
3,48,138,214,0,108,1.5
4,54,150,195,0,122,0.0


In [48]:
z_score = (df[cols] - mean)/std_dev
z_score.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak
0,-1.432359,0.410685,0.824621,-0.551041,1.382175,-0.831979
1,-0.478223,1.490940,-0.171867,-0.551041,0.753746,0.105606
2,-1.750404,-0.129442,0.769768,-0.551041,-1.524307,-0.831979
3,-0.584238,0.302660,0.138964,-0.551041,-1.131539,0.574398
4,0.051853,0.950812,-0.034736,-0.551041,-0.581664,-0.831979


In [49]:
mask = (np.abs(z_score) < 3).all(axis = 1)

In [104]:
df_clean = df[mask].reset_index(drop=True)
df_clean.head()
df_clean.shape

(899, 12)

In [51]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
dfle = df_clean
columns = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]
dfle[columns] = dfle[columns].apply(lambda x: le.fit_transform(x))
dfle.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0


In [52]:
X =  dfle.drop("HeartDisease", axis ="columns")
y = dfle["HeartDisease"]

In [53]:
dfle.HeartDisease.value_counts()

HeartDisease
1    492
0    407
Name: count, dtype: int64

In [55]:
407/492

0.8272357723577236

In [58]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[0:3]

array([[-1.42815446,  0.515943  ,  0.2245723 ,  0.46590022,  0.84963584,
        -0.5503622 ,  0.01414258,  1.38431998, -0.8229452 , -0.85546862,
         1.04249607],
       [-0.47585532, -1.93819859,  1.27063705,  1.63471366, -0.16812204,
        -0.5503622 ,  0.01414258,  0.7529728 , -0.8229452 ,  0.13751561,
        -0.62216462],
       [-1.7455875 ,  0.515943  ,  0.2245723 , -0.1185065 ,  0.79361247,
        -0.5503622 ,  1.60341473, -1.53566071, -0.8229452 , -0.85546862,
         1.04249607]])

In [61]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X_scaled,y, stratify=y, random_state=10)

In [65]:
y_train.value_counts()

HeartDisease
1    369
0    305
Name: count, dtype: int64

In [66]:
305/369

0.8265582655826558

In [88]:
X_train.shape

(674, 11)

In [89]:
X_test.shape

(225, 11)

In [87]:
from sklearn.svm import SVC

model = SVC(kernel="rbf", C=10,gamma="auto")
model.fit(X_train,y_train)
scores = model.score(X_test,y_test)
scores

0.8444444444444444

In [90]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(SVC(), X, y,cv = 5)
scores.mean()

np.float64(0.6884667908131595)

In [98]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    estimator= SVC(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
bag_model.fit(X,y)
bag_model.oob_score_

0.7174638487208009

In [99]:
scores = cross_val_score(bag_model,X,y,cv =5)
scores.mean()

np.float64(0.6873494723774053)

In [100]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

scores = cross_val_score(DecisionTreeClassifier(), X, y,cv = 5)
scores.mean()

np.float64(0.7462631905648665)

In [101]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 100,
    max_samples = 0.8,
    oob_score = True,
    random_state = 0
)
bag_model.fit(X,y)
bag_model.oob_score_

0.8509454949944383

In [102]:
scores = cross_val_score(bag_model,X,y,cv =5)
scores.mean()

np.float64(0.8152513966480447)

In [103]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(), X, y, cv=5)
scores.mean()

np.float64(0.8285971446306644)